In [ ]:
%matplotlib inline

import os
import numpy as np
import pickle
import matplotlib.pyplot as plt

import matplotlib.animation as animation
import scipy.io as sio

default_dir = os.path.dirname(os.getcwd())

os.chdir(default_dir)

from scripts import celegans_dynome as c_dynome
from scripts import celegans_utilities as c_util
from scripts import sys_paths as paths
from timeit import default_timer as timer

In [ ]:
neurons_list = c_dynome.neurons_list
neurons_name=[]
for i in range(0,279):
    neurons_name.append(neurons_list[i]['name'].encode('ascii','ignore'))

sensory = []
inter = []
motor = []

for neuron in neurons_list:
    
    if neuron['group'] == 'sensory':
        
        sensory.append(neuron['index'])
        
    elif neuron['group'] == 'inter':
        
        inter.append(neuron['index'])
        
    else:
        
        motor.append(neuron['index'])

In [ ]:
from anytree import Node, RenderTree
from anytree.dotexport import RenderTreeGraph
from graphviz import Digraph
# graphviz needs to be installed for the next line!
import operator

In [ ]:
pkl_file = open('weighted_sum.pkl', 'rb')
data1 = pickle.load(pkl_file)
cpt=np.reshape(data1,(279,279))
def extend_tree(new_parent,pindex,plist,count):  
    if (count>1):
        count=0
        return
    
    if new_parent==None:
        return
    elif pindex not in plist:
        plist.append(pindex)
        child_list={}
        count=count+1
         
            
        for i in range(0,279):
            prob=cpt[pindex,i]
            if pindex in sensory:
                if (prob>=0.1 and i not in plist and i in inter):
                    child_list[i]=prob
            if pindex in inter:
                if (prob>=0.1 and i not in plist and i in motor):
                    child_list[i]=prob

        if child_list==None:
            return
        
        else:
            sorted_child=sorted(child_list.items(), key=operator.itemgetter(1))
            num=len(sorted_child)
            for j in range(0,min(num,num)):
                c=sorted_child[num-j-1][0]
                new_child=Node(neurons_name[c],parent=new_parent)
                extend_tree(new_child,c,plist,count)
        

In [ ]:
## example: PLML as the input node

count=0
inter_layer=1
active=[]
Input=Node("Input")


PLML=Node("PLML",parent=Input)
extend_tree(PLML,278,active, count)


for pre, fill, node in RenderTree(Input):
    print("%s%s" % (pre, node.name))

RenderTreeGraph(Input).to_picture("PLML.png")
